<a href="https://colab.research.google.com/github/Phionanamugga/Autonomous_Navigation/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
import time
import supervision as sv

In [3]:
# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Pre-trained YOLOv8 nano model

In [4]:
# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, nn_budget=100, nms_max_overlap=1.0)

In [5]:
# Initialize video capture (replace with your video file or camera)
cap = cv2.VideoCapture('sample_traffic_video.mp4')